# High-level Keras R (TF) CNN Example

In [18]:
# SETUP
#
# Install keras R
# install.packages('keras')
# 
# Update reticulate from cran (it defaults to mran which has an outdated version)
# install.packages("reticulate", repos = "https://cloud.r-project.org")
#
# Install R.matlab, if we're using .mat files for data
# install.packages('R.matlab')

In [44]:
library(keras)
use_python('/anaconda/envs/py35')

# Import util functions
source("./common/utils.R")

# Import hyper-parameters
params <- load_params("cnn")

In [45]:
# Performance Improvement
# 1. Make sure channels-first (not last)
backend()$set_image_data_format('channels_first')

In [46]:
# py_config()
print(paste0("OS: ", Sys.info()["sysname"]))
print(R.version$version.string)
print(paste0("Keras: ", packageVersion("keras")))
print(paste0("Tensorflow: ", packageVersion("tensorflow")))
print(paste0("Keras using ", backend()$backend()))
print(paste0("Keras channel ordering is ", backend()$image_data_format())) 
print(paste0("GPU: ", get_gpu_name()))
print(get_cuda_version())
print(get_cudnn_version())

[1] "OS: Linux"
[1] "R version 3.4.1 (2017-06-30)"
[1] "Keras: 2.0.6"
[1] "Tensorflow: 1.3.1"
[1] "Keras using tensorflow"
[1] "Keras channel ordering is channels_first"
[1] "GPU: Tesla K80"
[1] "CUDA Version 8.0.61"
[1] "CuDNN Version 6.0.21"


In [22]:
create_symbol <- function(n_classes = params$N_CLASSES){
    
    # initialize a sequential model
    model <- keras_model_sequential() %>%
    
    # 2D convolutional layers being fed 32x32 pixel images   
    layer_conv_2d(filters = 50, kernel_size = c(3, 3), padding = "same", activation = "relu", input_shape = c(3, 32, 32)) %>%
    layer_conv_2d(filters = 50, kernel_size = c(3, 3), padding = "same", activation = "relu") %>%
    
    # max pooling
    layer_max_pooling_2d(pool_size = c(2, 2), strides = c(2, 2)) %>%
    layer_dropout(0.25) %>%

    # 2D convolutional layers 
    layer_conv_2d(filters = 100, kernel_size = c(3, 3), padding = "same", activation = "relu") %>%
    layer_conv_2d(filters = 100, kernel_size = c(3, 3), padding = "same", activation = "relu") %>%
    
    # max pooling
    layer_max_pooling_2d(pool_size = c(2, 2), strides = c(2, 2)) %>%
    layer_dropout(0.25) %>%

    # flatten into feature vector
    layer_flatten() %>%
    layer_dense(512, activation = "relu") %>%
    layer_dropout(0.5) %>%
    layer_dense(n_classes, activation = "softmax")  
    
    return(model)
}

In [23]:
init_model <- function(m, lr=params$LR, momentum=params$MOMENTUM){
    m %>% compile(
      loss = "categorical_crossentropy",
      optimizer = optimizer_sgd(lr, momentum),
      metrics = "accuracy"
    )
    return(m)
}

In [24]:
# Data Preparation 
cifar <- cifar_for_library(one_hot = TRUE)
x_train <- cifar$x_train
y_train <- cifar$y_train
x_test <- cifar$x_test
y_test <- cifar$y_test

rm(cifar)

[1] "Preparing train set ..."
[1] "Preparing test set ..."


In [25]:
cat('x_train shape:', dim(x_train), '\n')
cat('x_test shape:', dim(x_test), '\n')
cat('y_train shape:', dim(y_train), '\n')
cat('y_test shape:', dim(y_test), '\n')

x_train shape: 50000 3 32 32 
x_test shape: 10000 3 32 32 
y_train shape: 50000 10 
y_test shape: 10000 10 


In [26]:
# Load symbol
sym = create_symbol()

In [27]:
# Initialise model
model = init_model(sym)

In [ ]:
# summary(model)

In [28]:
# Main training loop
model %>% fit(
    x_train, y_train,
    batch_size = params$BATCHSIZE,
    epochs = params$EPOCHS,
    verbose = 1)

In [29]:
# Main evaluation loop
y_guess <- model %>% predict_classes(x_test, batch_size = params$BATCHSIZE)
y_truth <- apply(y_test, 1, function(x) which.max(x)-1)

In [30]:
print(paste0("Accuracy: ", sum(y_guess == y_truth)/length(y_guess)))

[1] "Accuracy: 0.7729"
